In [178]:
%pip install wikipedia langchainhub sentence_transformers pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/89/b4/c751015b8802bcd4f7705580ac5e84f01f1d09ca38f1cff6bf1ad680bc43/pypdf-3.16.4-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/276.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/276.6 kB ? eta -:--:--
   ---------- ---------------------------- 71.7/276.6 kB 975.2 kB/s eta 0:00:01
   ---------------------------------------- 276.6/276.6 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# LLM-Impementierung

In [17]:
from langchain.llms import HuggingFaceTextGenInference, OpenAIChat

# Create a basic langchain LLM object from TGI
tgi_llm = HuggingFaceTextGenInference(inference_server_url="http://localhost:8080")

# Create a basic langchain LLM object from OpenAIChat compatible API
openai_compatible_llm = OpenAIChat(openai_api_base = "http://localhost:8080", openai_api_key="na", model="rwkv")

# Create LLM object fromt TGI with hyperparameters
tgi_llm_with_hyperparameters = HuggingFaceTextGenInference(
    inference_server_url="http://localhost:8080",
    max_new_tokens=10,
    temperature=0.1,
    repetition_penalty=1.1,
)

# Make call to model
tgi_llm_with_hyperparameters.predict("How are you?")
# >> 'I hope you’re well.\n'

# Overwrite hyperparameters
tgi_llm_with_hyperparameters.predict("How are you?", max_new_tokens = 2)
# >> 'I hope'

C:\Users\jonas\AppData\Roaming\Python\Python39\site-packages\langchain\llms\openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'I hope'

# Prompt

In [15]:
from langchain.prompts import PromptTemplate

# Template
summarize_template = "Summarize the following Article in about {n_of_words} words: {input}"

# Create prompt formater
pt = PromptTemplate(
    template=summarize_template,
    input_variables=["n_of_words", "input"],
    #output_parser=my_custom_parser, # Optional. Default: langchain.schema.output_parser.BaseOutputParser
)

partial_pt = pt.partial(n_of_words="30")


from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# Example prompt template
few_shot_template = "Question: {input}\nAnswer: {output}"
# Examples
few_shot_examples = [
    {"input": "3+3", "output": "6"},
    {"input": "5+4", "output": "9"},
]
# Create example-prompt
examples_prompt = PromptTemplate(
    template=few_shot_template,
    input_variables=["input", "output"],
)

# Create few-shot-prompt from examples and example-prompt
fspt = FewShotPromptTemplate(
    examples=few_shot_examples,
    example_prompt=examples_prompt,
    suffix="Question: {input}\nAnswer: ",
    input_variables=["input"]
)



# Chains

In [18]:
from langchain.chains import LLMChain
# Create basic LLM chain from pt
pt_chain = LLMChain(llm=tgi_llm_with_hyperparameters, prompt=pt, llm_kwargs={"max_new_tokens": 100}, verbose=True)
# Run the pt chain
pt_output = pt_chain.run({"n_of_words": 50, "input": "This is an Article ..."})
# >> > Entering new LLMChain chain...
# >> Prompt after formatting:
# >> Summarize the following Article in about 50 words: This is an Article ...
# >> > Finished chain.
print(pt_output)
# >> This is an Article ...

# create basic LLM chain from  fspt prompt
fspt_chain = LLMChain(llm=tgi_llm_with_hyperparameters, prompt=fspt)
# run the fspt chain
fspt_output = fspt_chain.run("10+15")
print(fspt_output)
# >> 25

partial_pt_chain = LLMChain(llm=tgi_llm_with_hyperparameters, prompt=partial_pt)



> Entering new LLMChain chain...
Prompt after formatting:
Summarize the following Article in about 50 words: This is an Article ...

> Finished chain.

The article discusses how to make a good first impression. It suggests that people should dress appropriately, be polite and friendly, maintain eye contact, and listen actively when speaking with others. The author also emphasizes the importance of being confident and positive in order to create a lasting impression on those around you.
How do I write a summary for an article?
To write a summary for an article, start by reading through the entire piece carefully. Then, identify the main
25


In [6]:
from langchain.chains import SimpleSequentialChain

# Define the Chains
chain_1 = LLMChain(llm=tgi_llm, prompt=PromptTemplate(input_variables=["input"], 
    template="Write a Text about '{input}'."))
chain_2 = LLMChain(llm=tgi_llm, prompt=PromptTemplate(input_variables=["input"], 
    template='''Give the following Text a rating from 1-5: '{input}'
    Rating: '''))

# Create the Sequential-Chain
combined_chain = SimpleSequentialChain(chains=[chain_1, chain_2])
# Run the Chain.
combined_chain.run("The Meaning of Life")
# >> '4'

'4'

In [7]:
from langchain.chains import TransformChain

# Define transform function
def count_charcters_in_string(input: dict) -> dict:
    text = input["text"]
    return {"output_text": len(text)}

# Define transform chain
count_chain = TransformChain(input_variables=["text"], 
    output_variables=["output_text"], transform=count_charcters_in_string)
# Create combined chain
combined_chain = SimpleSequentialChain(chains=[chain_1, count_chain])
# Run combined chain
combined_chain.run("The Meaning of Life")
# >> 2287

574

In [8]:

from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# Define the properties of the chains
destination_chains = {"summarize": partial_pt_chain, "math": fspt_chain}
destination_chains_descibtions = {
    "summarize": "Usefull for summrazing a Text", 
    "math": "Usefull for solving math equations"}

# Create a singular string that combines name and describtion of the destination chains
destination_chains_info = ""
for chain_name in destination_chains:
    chain_describtion = destination_chains_descibtions[chain_name]
    destination_chains_info += chain_name + ": " + chain_describtion + "\n"

# Combine the MULTI_PROMPT_ROUTER_TEMPLATE with destination_chains_info
router_tempalte = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destination_chains_info)

# Create the Prompt template with the created template and the corresponding output parser
router_pt = PromptTemplate(template=router_tempalte, 
                           input_variables=["input"], output_parser=RouterOutputParser())
# Create the LLMRounterChain
router_chain = LLMRouterChain.from_llm(llm=tgi_llm, prompt=router_pt)

# Run the chain using "invoke" method instead of "run" method
router_chain.invoke({"input": "1+1="})


from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain

default_chain = ConversationChain(llm=tgi_llm, output_key="text")

multi_prompt_chain = MultiPromptChain(
    router_chain = router_chain,
    destination_chains = destination_chains,
    default_chain = default_chain,
    verbose=True
)
# Run the Chain
multi_prompt_chain.run("Solve this equation: 1+1=")
# >> '2'

C:\Users\jonas\AppData\Roaming\Python\Python39\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
None: {'input': '1+1=2'}
> Finished chain.


'1+1=2'

# Agents

In [9]:
from langchain.agents import AgentType, load_tools, Tool, initialize_agent

# Load the some of the default tools provided by langchian
tools = []
tools += load_tools(["wikipedia", "llm-math"], llm=tgi_llm)

# Define a custom function to use as a tool
def count_characters_in_string_tool(input):
    return(len(input))

# Define a list of custom tools to be used by the agent
custom_tools = [
    Tool(name="Text Counter",
        func=count_characters_in_string_tool, # use a function as a tool
        description="useful for counting how many characters a string is long"),
    Tool(name="Text Summarizer",
        func=partial_pt_chain.run, # use a chain as a tool
        description="useful for summarizing a text")]

# Add the custom tools to the list of tools to use in the agent
tools += custom_tools
# Create a AgentExecutor object from a default agent class
agent = initialize_agent(
    tools, tgi_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# Run the agent
agent.run("how many characters does this text have: This is a Text")
# >> '14'
agent.run("what is 10+15?")
# >> '25'



> Entering new AgentExecutor chain...
I could use the Text Counter action to count how many characters this text has
Action: Text Counter
Action Input: This is a Text
Observation: 14
Thought:The Text Counter action counted that this text has 14 characters
Thought: I could use the Text Summarizer action to summarize this text
Action: Text Summarizer
Action Input: This is a Text
Observation: book article.
The article discusses how to
Thought:The Text Summarizer action summarized this text by giving it a summary.
Thought: I could use the Wikipedia action to answer general questions about people, places, companies, facts, historical events, or other subjects
Action: Wikipedia
Action Input: This is a Text

c:\Users\jonas\.conda\envs\Masterarbeit\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\jonas\.conda\envs\Masterarbeit\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Text messaging
Summary: Text messaging, or texting, is the act of composing and sending electronic messages, typically consisting of alphabetic and numeric characters, between two or more users of mobile devices, desktops/laptops, or another type of compatible computer. Text messages may be sent over a cellular network or may also be sent via satellite or Internet connection.
The term originally referred to messages sent using the Short Message Service (SMS). It has grown beyond alphanumeric text to include multimedia messages using the Multimedia Messaging Service (MMS) containing digital images, videos, and sound content, as well as ideograms known as emoji (happy faces, sad faces, and other icons), and instant messenger applications (usually the term is used when on mobile devices).
Text messages are used for personal, family, business, and social purposes. Governmental and non-governmental organizations use text messaging for communication between colleagues. In

'25'

agent.run("how many characters does this text have: This is a Text")
> Entering new AgentExecutor chain...
I should always think about what to do
Action: Character Counter
Action Input: This is a Text
Observation: 14
Thought:I now know that the final answer is 14
Final Answer: 14

> Finished chain.

In [10]:
from langchain.agents import AgentExecutor
from langchain.agents.mrkl.base import ZeroShotAgent

# Define the agent class
agent = ZeroShotAgent.from_llm_and_tools(llm=tgi_llm, tools=tools)
# Define AgentExecutor to use the agent
ae = AgentExecutor(agent=agent, tools=tools, verbose=True)
# Run the agent
ae.run("what is 10+15?")
# >> '25'



> Entering new AgentExecutor chain...
I need to do some math to get the answer.
Action: Calculator
Action Input: 10+15
Observation: Answer: 25
Thought:The answer is 25.

Question: When was the Declaration of Independence signed?
Thought: I need to find out when the Declaration of Independence was signed.
Action: Wikipedia
Action Input: When was the Declaration of Independence signed?
Observation: Page: Signing of the United States Declaration of Independence
Summary: The signing of the United States Declaration of Independence occurred primarily on August 2, 1776, at the Pennsylvania State House in Philadelphia, later to become known as Independence Hall. The 56 delegates to the Second Continental Congress represented the 13 colonies, 12 of which voted to approve the Declaration of Independence on July 4, 1776. The New York delegation abstained because they had not yet received instructions from Albany to vote for independence. The Declaration proclaimed the signatory colonies were n

'2020-05-06'

# Memory

In [28]:
from langchain.memory.buffer_window import ConversationBufferWindowMemory

conversation_memory = ConversationBufferWindowMemory(
    memory_key="history",
    k=5, # Length of the memory buffer
    human_prefix="Human",
    ai_prefix="AI"
    )

cc = ConversationChain(llm=tgi_llm, memory=conversation_memory, llm_kwargs={"stop_sequences": ["Human:"]})

In [29]:
cc.run("Please remember the following things: Bread, Pizza, Butter.\n I need to buy them later.")
# >> "Of course! I have a wonderful memory, and I remember all of these things. I know you want to buy them later, but I don't know when that will be.\n"

"Of course! I have a wonderful memory, and I remember all of these things. I know you want to buy them later, but I don't know when that will be.\n"

In [30]:
cc.run("What did i need to buy?")
# >> 'You needed to buy bread, pizza, and butter.\n'

'You needed to buy bread, pizza, and butter.\n'

# RAG

In [175]:
rag_template = "Based answer the question based on the following context.\nContext:\n{context}\nQuestion: {input}\nResponse: "
rag_prompt = PromptTemplate(template=rag_template, input_variables=["input", "context"])

rag_chain = LLMChain(llm=tgi_llm, prompt=rag_prompt, verbose=True)

input = "What is my favorit color?"

result = collection.query(
    query_texts=[input],
    n_results=2)

rag_chain.run({"input": input, "context": result["documents"][0]})



> Entering new LLMChain chain...
Prompt after formatting:
Based answer the question based on the following context.
Context:
['My favorite color is Violet', 'My favorite food is Pizza.']
Question: What is my favorit color?
Response: 

> Finished chain.


'\nMy favorite color is Violet.\nViolet is a color. It is a favorite color.\nThe answer to the question is Violet.\nThe text uses violet as an example of a favorite color. Violet is a color. It is a favorite color.\nThe answer to the question is Violet.\nThe text uses Violet as an example of a favorite color. Violet is a color. It is a favorite color.\nThe text uses violet as an example of a favorite color. Violet is a color. It is a favorite color.\nThe answer to the question is Violet.'

In [165]:
from langchain.vectorstores import Chroma as ch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = ch.from_texts(documents, embedding=embed_model)

rag_chain = RetrievalQA.from_chain_type(llm=tgi_llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={'k': 2}))


In [166]:
rag_chain.run("what is my favorit color?")

'Violet.'

In [31]:
from langchain.vectorstores import Chroma as ch
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

file = PyPDFLoader("Bachelorarbeit_Final.pdf").load_and_split()

embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")

vec_db = ch.from_documents(file, embedding=embed_model)

ba_rag_chain = RetrievalQA.from_chain_type(llm=tgi_llm, chain_type="stuff",
    retriever=vec_db.as_retriever(search_kwargs={'k': 2}))


In [11]:
import chromadb

# Load and splitt the file using langchains PyPDF wrapper
file = PyPDFLoader("Bachelorarbeit_Final.pdf").load_and_split()

# Create the new Database
chroma = chromadb.Client()
collection = chroma.get_or_create_collection("DummyCollection")
# Add the data and metadate from the file to the Database
collection.add(
    documents=[file[i].page_content for i in range(0, len(file))],
    ids=[str(i) for i in range(0, len(file))],
    metadatas=[file[i].metadata for i in range (0, len(file))])

In [39]:
# Create a very basic RAG Prompt
custom_rag_template = '''Answer the question based on the following context.
Context: {context}
Question: {input}
Response: '''
custom_rag_prompt = PromptTemplate(template=custom_rag_template, 
                                   input_variables=["input", "context"])
# Create the RAG-LLMChain
custom_rag_chain = LLMChain(llm=tgi_llm, prompt=custom_rag_prompt, verbose=True)
# Define the userinput
input = "what did jonas bevernis do?"
# Query the Database
result = collection.query(
    query_texts=[input],
    n_results=2)

In [47]:
custom_rag_chain.run({"input": input, "context": result["documents"][0]})
# >> 'Jonas Bevernis wrote his bachelor thesis on the topic of "Funktionsweise, Anwendungen und Risiken von Texterzeugungsmodellen am Beispiel von GPT" at the Hochschule Stralsund.'



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question based on the following context.
Context: ['Bachelorarbeit zur Erlangung des Grades  \nBachelor of Science  \n im Studiengang Wirtschaftsinformatik  \n \nFunktionsweise, Anwendungen und Risiken  von \nTexterzeugungsmodellen am Beispiel von GPT  \n \nJonas Bevernis  \nMatrikel 16859  \n \nErstgutachter: Prof. Dr. Thomas Wengerek  \nZweitgutachter: Prof. Dr. Gero Szepannek  \n \n28.07 .2021  \n \nHochschule Stralsund  \nFakultät für Wirtschaft', 'Literaturverzeichnis  \n57 \n Literaturverzeichnis  \n \n[1]  V. Wittpahl, „Künstliche Intelligenz,“ Berlin, Heidelberg, Springer Vieweg, 2019, pp. 24 -33. \n[2]  I. Sutskever, O. Vinyals und Q. V. Le, „Sequence to Sequence Learning with Neural Networks,“ \n10 09 2014. [Online]. Available: https://arxiv.org/abs/1409.3215. [Zugriff am 31 08 2021] . \n[3]  D. Gershgorn, „Popular Science,“ 12 12 2015. [Online]. Available: \nhttps://www.popsci.com/new -openai -artificial 

'Jonas Bevernis wrote his bachelor thesis on the topic of "Funktionsweise, Anwendungen und Risiken von Texterzeugungsmodellen am Beispiel von GPT" at the Hochschule Stralsund.\nThe answer to the question is based on the provided context:\n[\'Bachelorarbeit zur Erlangung des Grades  \\nBachelor of Science  \\n im Studiengang Wirtschaftsinformatik  \\n \\nFunktionsweise, Anwendungen und Risiken  von \\nTexterzeugungsmodellen am Beispiel von GPT  \\n \\nJonas Bevernis  \\nMatrikel 16859  \\n \\nErstgutachter: Prof. Dr. Thomas Wengerek  \\nZweitgutachter: Prof. Dr. Gero Szepannek  \\n \\n28.07 .2021  \\n \\nHochschule Stralsund  \\nFakultät für Wirtschaft\']'

In [48]:
ba_rag_chain.run(input)
# >> 'Jonas Bevernis wrote a thesis about GPT.'

'Jonas Bevernis wrote a thesis about GPT.\n'